In [4]:
import os
import requests
import json

In [6]:
!pip install diskcache
!pip install langchain
from diskcache import Cache
from langchain.text_splitter import CharacterTextSplitter

In [8]:
os.environ["CLAUDE_API_KEY"] = "DSI_Claude_API_key" #replace with the API key of the DSI account
claude_api_key = os.getenv("CLAUDE_API_KEY")

In [10]:
cache = Cache('/tmp/claude_cache')

In [12]:
def query_claude(query, context=""):
    url = "https://api.anthropic.com/v1/messages/batches"  

    headers = {
        "Authorization": f"Bearer {claude_api_key}",
        "Content-Type": "application/json",
    }
    payload = {
        "query": query,
        "context": context,
    }

    response = requests.post(url, headers=headers, data=json.dumps(payload))

    if response.status_code == 200:
        return response.json().get("response")  # Adjust based on actual API response format
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None


In [14]:
def query_claude_with_cache(query, context=""):

    cache_key = f"{query}-{context}"
    
    # Check if response is already in cache
    if cache_key in cache:
        print("Using cached response.")
        return cache[cache_key]
    
    response = query_claude(query, context)
    if response:
        cache[cache_key] = response
    return response

In [16]:
def split_text_into_chunks(text, max_length=1000):
    text_splitter = CharacterTextSplitter(separator=" ", chunk_size=max_length, chunk_overlap=100)
    chunks = text_splitter.split_text(text)
    return chunks


In [18]:
def process_long_document_with_claude(text):
    chunks = split_text_into_chunks(text)
    responses = []
    
    for i, chunk in enumerate(chunks):
        # Query each chunk with caching
        response = query_claude_with_cache(f"Chunk {i}", context=chunk)
        responses.append(response)
    
    return responses

In [20]:
def extract_text_from_pdf(pdf_path):
    """
    Extracts all text from a PDF file.
    """
    text = ""
    with fitz.open(pdf_path) as pdf:
        for page_num in range(len(pdf)):
            page = pdf[page_num]
            text += page.get_text("text")  # Extract text from each page
    return text

In [39]:
!pip install pymupdf

In [22]:
import fitz

In [24]:
pdf_path = "/Users/xuanchen99/Desktop/climate policy/Jacksonville, FL - Resilient Jacksonville (Oct. 2023).pdf"


In [26]:
long_text = extract_text_from_pdf(pdf_path)

In [ ]:
responses = process_long_document_with_claude(long_text)

In [ ]:
for response in responses:
    print(response)